<a href="https://colab.research.google.com/github/biybiyy29-lgtm/UTS23B_PROGJAR_23423015./blob/main/UTS23B_PROGJAR_23423015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soal **1**

In [ ]:
%%writefile server.py
import socket

# Membuat socket TCP
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind ke alamat dan port 5050
server_socket.bind(("0.0.0.0", 5050))

# Listen untuk koneksi masuk
server_socket.listen(1)
print("Server berjalan di port 5050...")

# Menunggu client terhubung
conn, addr = server_socket.accept()
print(f"Client terhubung dari {addr}")

while True:
    # Terima pesan dari client
    data = conn.recv(1024)

    if not data:
        break

    # Tampilkan di server
    print("Pesan diterima:", data.decode())

    # Kirim kembali (echo) ke client
    conn.sendall(data)

conn.close()
server_socket.close()



Overwriting server.py


In [ ]:
%%writefile client.py
import socket

# Membuat socket TCP
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Terhubung ke server di localhost port 5050
client_socket.connect(("127.0.0.1", 5050))

# Mengirim pesan ke server
pesan = "Tes Koneksi"
client_socket.sendall(pesan.encode())

# Menerima balikan dari server
data = client_socket.recv(1024)

print("Hasil balikan dari server:", data.decode())

client_socket.close()


Overwriting client.py


In [ ]:
!python server.py &

Traceback (most recent call last):
  File "/content/server.py", line 7, in <module>
    server_socket.bind(("0.0.0.0", 5050))
OSError: [Errno 98] Address already in use


In [ ]:
!python client.py

Traceback (most recent call last):
  File "/content/client.py", line 7, in <module>
    client_socket.connect(("127.0.0.1", 5050))
TimeoutError: [Errno 110] Connection timed out


# Soal **2**

In [ ]:
%%writefile server.py
import socket
import threading

# List untuk menyimpan semua client
clients = []

# Fungsi broadcast pesan ke semua client lain
def broadcast(message, sender_conn):
    for client in clients:
        if client != sender_conn:
            try:
                client.send(message)
            except:
                pass

# Fungsi untuk menangani setiap client
def handle_client(conn, addr):
    print(f"Client terhubung: {addr}")
    while True:
        try:
            msg = conn.recv(1024)
            if not msg:
                break
            print(f"Dari {addr}: {msg.decode()}")
            broadcast(msg, conn)
        except:
            break

    conn.close()
    clients.remove(conn)
    print(f"Client keluar: {addr}")

# Setup server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(("0.0.0.0", 5050))
server_socket.listen(5)

print("Server chat berjalan di port 5050...")

while True:
    conn, addr = server_socket.accept()
    clients.append(conn)

    thread = threading.Thread(target=handle_client, args=(conn, addr))
    thread.start()


Overwriting server.py


In [ ]:
%%writefile client.py
import socket
import threading

def terima_pesan(sock):
    while True:
        try:
            msg = sock.recv(1024)
            if msg:
                print("Pesan masuk:", msg.decode())
        except:
            break

# Buat koneksi ke server
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(("127.0.0.1", 5050))

# Thread untuk menerima pesan
thread = threading.Thread(target=terima_pesan, args=(client_socket,))
thread.daemon = True
thread.start()

# Loop untuk mengirim pesan
while True:
    pesan = input("")
    client_socket.send(pesan.encode())


Overwriting client.py


In [ ]:
%%writefile server.py
%%writefile client.py

Overwriting server.py


In [ ]:
!python server.py &
!python client.py

  File "/content/server.py", line 1
    %%writefile client.py
    ^
SyntaxError: invalid syntax
object address  : 0x7ded8bd4b5e0
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


# Soal **3**

In [ ]:
import socket
import time

# ============================================
# Fungsi untuk membuat koneksi TCP ke server
# ============================================
def connect_to_server(host, port, timeout_connect=3):
    print("=== Mencoba koneksi ke server ===")
    print(f"Host : {host}")
    print(f"Port : {port}")
    print(f"Timeout koneksi : {timeout_connect} detik\n")

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Set timeout untuk connect
    client.settimeout(timeout_connect)

    try:
        start = time.time()
        client.connect((host, port))
        end = time.time()
        print(f"✔ Berhasil terhubung ke server dalam {end - start:.2f} detik\n")
        return client

    except socket.timeout:
        print("Koneksi timeout saat mencoba connect!")
        return None
    except Exception as e:
        print(f"Error lain saat connect: {e}")
        return None


# =======================================================
# Fungsi untuk mengirim dan menerima data dari server
# =======================================================
def communicate_with_server(client, timeout_read=2):
    if client is None:
        print("\n⚠ Client tidak terhubung. Tidak dapat kirim data.")
        return

    client.settimeout(timeout_read)
    print(f"=== Mode komunikasi ===")
    print(f"Timeout membaca data: {timeout_read} detik\n")

    try:
        while True:
            pesan = input("Ketik pesan (atau 'exit' untuk keluar): ")

            if pesan.lower() == "exit":
                print("Menutup koneksi...")
                break

            # Mengirim pesan
            client.send(pesan.encode())
            print("Pesan terkirim, menunggu balasan server...")

            try:
                data = client.recv(1024)
                print("Balasan server:", data.decode(), "\n")

            except socket.timeout:
                print("Koneksi timeout saat membaca data!\n")

    except Exception as e:
        print(f"Error komunikasi: {e}")

    finally:
        client.close()
        print("\n=== Koneksi ditutup ===")


# ================================
# Program utama (main)
# ================================
if __name__ == "__main__":
    host = "127.0.0.1"
    port = 5000

    # 1. Coba koneksi ke server (timeout 3 detik)
    client = connect_to_server(host, port, timeout_connect=3)

    # 2. Jika berhasil connect → lanjut komunikasi (timeout recv 2 detik)
    communicate_with_server(client, timeout_read=2)

=== Mencoba koneksi ke server ===
Host : 127.0.0.1
Port : 5000
Timeout koneksi : 3 detik

Error lain saat connect: [Errno 111] Connection refused

⚠ Client tidak terhubung. Tidak dapat kirim data.
